# Project 1, Part 5, Data Visualization



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

import json

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

import ipywidgets as widgets
from IPython.display import display

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
gmaps.configure(api_key="")

# 1.5 Example of a Data Visualization created using Python

The data science team would like for you to create an example of a data visualization using Python from data in a Pandas dataframe containing data from an SQL query.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  Any query of your choosing.  You can write a query from scratch.  You can use a query from a previous problem in this project.  You can use a query from the labs.  The idea is to come up with a query whose resulting data will make for an excellent quality data visualization. 

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.  Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.

Once you have the data in a Pandas dataframe, you may write as much Python code and use as many code cells as you wish to produce the data visualization.

You may only use Python modules that are currently installed in the Anaconda Docker container.  You may not install additional modules or any other software.

All work must be done in Docker in your VM in AWS.  You may not use any external data visualization systems, such as Tableau, etc.

You may use any code from the labs to pattern your code after, however for the data visualization, you cannot wholesale copy a data visualization from the labs.

Ensure that it is properly titled, including titles for axes if present.

Ensure that when you check this Juptyer Notebook into GitHub that the data visualization is clearly visible.  

If you want to use Google Maps for your data visualization, this is fine, however, the image will not show up in GitHub.  So, just save the image to an image file, include it in the repo, and add a markdown cell to display the image file. Also with Google Maps, do NOT check gmap_api_key.txt into GitHub for security reasons.  If the grader needs to run it, they will supply their own gmap_api_key.txt.


In [5]:
# Use as many code cells as you need to create and display your data visualization


In [6]:
rollback_before_flag = True
rollback_after_flag = True

query = """

with first as (
    select 
        sa.sale_id, 
        sa.total_amount, 
        li.quantity, 
        li.product_id, 
        sa.store_id, 
        sa.sale_date
    from sales sa
        join line_items li
            on sa.sale_id = li.sale_id
),
second as(
    select 
        pr.description, 
        fi.store_id, 
        fi.sale_date, 
        avg(fi.total_amount / fi.quantity) as average_daily_sales_price
    from products pr
        join first fi 
            on pr.product_id = fi.product_id
    where
        fi.sale_date between '2020-02-07' and '2020-02-20'
group by
        1,
        2,
        3
),
third as (
    select 
        pr.description, 
        fi.store_id, 
        avg(fi.total_amount / fi.quantity) as average_sales_price
    from products pr
        join first fi 
            on pr.product_id = fi.product_id
    where
        fi.sale_date between '2020-02-07' and '2020-02-20'
    group by
        1,
        2
),
fourth as(
    select         
        se.description, 
        se.store_id, 
        se.sale_date, 
        se.average_daily_sales_price,
        th.average_sales_price
    from second se
        join third th
            on se.store_id = th.store_id
            and se.description = th.description
),
fifth as(
    select 
        fo.description, 
        fo.store_id, 
        st.latitude,
        st.longitude,
        fo.sale_date, 
        fo.average_daily_sales_price,
        fo.average_sales_price,
        average_daily_sales_price / average_sales_price as ratio_actual_expected
    from fourth fo
        join stores st
            on fo.store_id = st.store_id
)
select *
from fifth
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,description,store_id,latitude,longitude,sale_date,average_daily_sales_price,average_sales_price,ratio_actual_expected
0,Brocolli Stir Fry,1,37.8555,-122.2604,2020-02-07,56.237031,56.111881,1.002230
1,Brocolli Stir Fry,1,37.8555,-122.2604,2020-02-08,54.177690,56.111881,0.965530
2,Brocolli Stir Fry,1,37.8555,-122.2604,2020-02-09,55.203093,56.111881,0.983804
3,Brocolli Stir Fry,1,37.8555,-122.2604,2020-02-10,57.398427,56.111881,1.022928
4,Brocolli Stir Fry,1,37.8555,-122.2604,2020-02-11,56.011464,56.111881,0.998210
...,...,...,...,...,...,...,...,...
555,Tilapia Piccata,5,36.1568,-86.7881,2020-02-16,57.887814,59.314400,0.975949
556,Tilapia Piccata,5,36.1568,-86.7881,2020-02-17,61.103448,59.314400,1.030162
557,Tilapia Piccata,5,36.1568,-86.7881,2020-02-18,55.930451,59.314400,0.942949
558,Tilapia Piccata,5,36.1568,-86.7881,2020-02-19,59.844505,59.314400,1.008937


In [7]:
def update_markers(selected_description, selected_date):
    #Filter the dataframe
    filtered_df = df[(df['description'] == selected_description) & (df['sale_date'] == selected_date)]
    
    #Cetner the map over the united states to see all updates
    center_location = (39.8283, -98.5795)  
    fig = gmaps.figure(center=center_location, zoom_level=3.8)

    # Define marker symbols based on the ratio_actual_expected values
    red_symbols = []
    yellow_symbols = []
    green_symbols = []
    for index, row in filtered_df.iterrows():
        ratio = row['ratio_actual_expected']
        location = (row['latitude'], row['longitude'])
        
        if ratio > 1:
            symbol = gmaps.Symbol(location=location, stroke_color="red", fill_color="red", scale=3)
            red_symbols.append(symbol)
        elif ratio < 1:
            symbol = gmaps.Symbol(location=location, stroke_color="green", fill_color="green", scale=3)
            green_symbols.append(symbol)
        else:
            symbol = gmaps.Symbol(location=location, stroke_color="yellow", fill_color="yellow", scale=3)
            yellow_symbols.append(symbol)

    # Create separate marker layers for each symbol color
    red_layer = gmaps.Markers(markers=red_symbols)
    green_layer = gmaps.Markers(markers=green_symbols)
    yellow_layer = gmaps.Markers(markers=yellow_symbols)
    
    # Remove existing layers before adding new ones
    fig.layers = []
    
    # Add the marker layers to the map
    fig.add_layer(red_layer)
    fig.add_layer(green_layer)
    fig.add_layer(yellow_layer)
    
    # Display the map and widgets
    display(fig)
    
    
# Create dropdown widgets for description and date
description_dropdown = widgets.Dropdown(
    options=df['description'].unique(),
    value=df['description'].unique()[0],
    description='Food Type:'
)

date_dropdown = widgets.Dropdown(
    options=df['sale_date'].unique(),
    value=df['sale_date'].unique()[0],
    description='Sale Date:'
)
ui = widgets.HBox([description_dropdown, date_dropdown])               

# Create an interactive output widget
output = widgets.interactive_output(update_markers, {'selected_description': description_dropdown, 'selected_date': date_dropdown})

# Display the map and widgets
display(ui, output)

Output()

# Explanation of the Plot Above
The plot is on the interesting side so I added this text bubble to explain the visual. The premise of the plot is it shows a plus and minus 7 day window around valentines day, comparing how the daily price of the meal compares to the average price of the meal over the window. This visual has widgets that let the user choose between meals and date to pick days around valentines day where the price of their desired meal is lower than average. The intended user is a partner who wants to plan a Valentine's Day date, but maybe wants to lean on the cheaper side. 

## Legend

### Red:
The meal is more expensive than average on this current date

### Yellow:
The meal is the average price on this current date

### Green:
The meal is less expensive than average on this current date

## Valentine's Day Food Pricing Map

![Cheapest Food Near Valentine's Day](Screenshot 2024-02-03 172624.png")
